# EBM DB

### Functions

In [1]:
import requests
import json  

def resolveURI(text):
    payload = { 'text': text, 'n': 1, 'ontology':'hfo' }
#   r = requests.post('http://ontomatch.lis.ic.unicamp.br/api/rest/resource', data=json.dumps(payload))
    r = requests.post('http://10.1.1.198:8080/OntoMatch/rest/resources', data=json.dumps(payload))
    
#     restReturn = r.text.replace('callback(', '')
#     restReturn = restReturn.replace('})', '}')
    returnJson = json.loads(r.text)[0]  

    return str({'uri': returnJson['uri'], 'similarity': returnJson['similarity'],'label': returnJson['label']})

print(resolveURI("chest pain"))

{'uri': 'http://bmi.utah.edu/ontologies/hfontology/C0008031', 'label': 'Chest Pain', 'similarity': 1.0}


## Connecting to google spreadsheet and generating csv

In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('resources/data/gdocs_client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("EBMDataCompilation").sheet1
print(sheet)

import pandas as pd

sheetData = sheet.get_all_values() 
df = pd.DataFrame(sheetData)
df.reset_index(drop=True, inplace=True)
df.to_csv('resources/data/EBMDataCompilation.csv', index=False)

<Worksheet 'Sheet1' id:od6>


# Verifying retrieved data 

In [3]:
rowsCount = sheet.row_count
colsCount =  sheet.col_count 
header = sheet.row_values(2)


# Extract and print all of the values
print("Lines: {0}".format(rowsCount))
print("Columns: {0}".format(colsCount))
print("Header: {0}".format(header))

Lines: 61
Columns: 19
Header: ['Disease', 'Source Type', 'Survey Ref', 'SurveyUrl', 'Article', 'Studies', 'n', ' n with disease', 'Group', 'Clinical Feature', 'CI LR+', 'LR+', 'LR+ floor', 'LR+ ceilling', 'CI LR-', 'LR-', 'LR- floor', 'LR- ceiling', 'Title']


## Connecting to neo4j db

In [4]:
from py2neo import Graph

### My parameters
#graph = Graph("http://neo4j.lis.ic.unicamp.br:80/db/data/", )
graph = Graph(host = "neo4j.lis.ic.unicamp.br", http_port = 80, bolt = False, secure = False)


if(graph):
    print ("Connected")
    print (graph)

graph.begin(autocommit = True)

Connected
<Graph uri='http://neo4j.lis.ic.unicamp.br:80/db/data/'>


## Generating the graph

In [5]:
import csv
from py2neo import Node, Relationship, NodeSelector

graph.delete_all()
selector = NodeSelector(graph)

with open('resources/data/EBMDataCompilation.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(csvfile)
    next(csvfile)
    next(csvfile)    
    for row in reader:
       
        disease = row[0]
        sourceType = row[1]
        surveyLabel = row[2]
        surveyUrl = row[3]
        articleLabel = row[4]
        numberOfStudies = row[5]
        a = row[6]
        a = row[7]
        groupLabel = row[8]
        clinicalFeatureLabel = row[9]
        
        cilrPositive  = row[10]
        lrPositive    = row[11]
        lrPositiveMin = row[12]
        lrPositiveMax = row[13]
        
        cilrNegative  = row[14]
        lrNegative    = row[15]
        lrNegativeMin = row[16]
        lrNegativeMax = row[17]
        
        surveytitle   = row[18]
        
        print("Disease: '"+disease+"' - URI: "+resolveURI(disease))
        clinicalFeatureUri = resolveURI(clinicalFeatureLabel)
        if (clinicalFeatureUri):
            print("clinicalFeatureLabel: '"+clinicalFeatureLabel+"' - URI: "+clinicalFeatureUri)
        
        #disease
        selected = selector.select("Disease", label=disease)
        diseaseNode= selected.first()

        if(diseaseNode):
            print("Found disease! ")
        else:
            diseaseNode = Node("Disease", label=disease)
            graph.create(diseaseNode)
    
        #survey
        selected =  selector.select("Article", label=surveyLabel)
        surveyNode= selected.first()

        if(surveyNode):
            print("Found Survey! ")
        else:
            surveyNode = Node("Article",type="survey", label=row[2], url= row[3], title=surveytitle)
            graph.create(surveyNode)
    
        #article
        selected =  selector.select("Article", label=articleLabel)
        articleNode= selected.first()

        if(articleNode):
            print("Found Article! ")
            print(articleLabel)
        else:
            if(articleLabel != "compilation"):
                articleNode = Node("Article",type="study", label=articleLabel)
                graph.create(articleNode)
                

        #group
        groupNode =  selector.select("ClinicalFeatureGroup", label=groupLabel).first()

        if(groupNode):
            print("Found group! ")
        else:
            if(groupLabel != "-"):
                groupNode = Node("ClinicalFeatureGroup", label=groupLabel)
                graph.create(groupNode)
                
                
        #clinicalfeature
        clinicalFeatureNode =  selector.select("ClinicalFeature", label=clinicalFeatureLabel).first()

        if(clinicalFeatureNode):
            print("Found clinicalfeature! ")
        else:
            if(clinicalFeatureLabel != "-"):
                clinicalFeatureNode = Node("ClinicalFeature", label=clinicalFeatureLabel)
                graph.create(clinicalFeatureNode)
                
        if(groupNode):
            clinicalFeature_clinicalFeatureGroup = Relationship(clinicalFeatureNode, "belongs_to", groupNode)
            graph.create(clinicalFeature_clinicalFeatureGroup)

        if(articleLabel == "compilation"):    
            clinicalFeature_survey = Relationship(clinicalFeatureNode, "mentioned_by", surveyNode, numberofStudies = numberOfStudies)
            graph.create(clinicalFeature_survey)
            disease_survey = Relationship(diseaseNode, "main_subject_of",surveyNode)
            graph.create(disease_survey)
        else:
            disease_article = Relationship(clinicalFeatureNode, "mentioned_by", articleNode)
            graph.create(disease_article)
            disease_survey = Relationship(diseaseNode, "main_subject_of",articleNode)
            graph.create(disease_survey)


            

        disease_clinicalfeature = Relationship(diseaseNode, "is_associated_with", clinicalFeatureNode, positiveLR = lrPositive, negativeLR = lrNegative )
        graph.create(disease_clinicalfeature)

    
        


Disease: 'Acute Coronary Syndrome' - URI: {'uri': 'http://bmi.utah.edu/ontologies/hfontology/C0948089', 'label': 'Acute Coronary Syndrome', 'similarity': 1.0}
clinicalFeatureLabel: 'Peripheral arterial disease' - URI: {'uri': 'http://bmi.utah.edu/ontologies/hfontology/C1704436', 'label': 'peripheral arterial disease', 'similarity': 1.0}
Disease: 'Acute Coronary Syndrome' - URI: {'uri': 'http://bmi.utah.edu/ontologies/hfontology/C0948089', 'label': 'Acute Coronary Syndrome', 'similarity': 1.0}
clinicalFeatureLabel: 'Pain similar to previous ischemia' - URI: {'uri': 'http://bmi.utah.edu/ontologies/hfontology/C0746731', 'label': 'Acute myocardial ischemia', 'similarity': 0.4242424242424242}
Found disease! 
Found Survey! 
Disease: 'Acute Coronary Syndrome' - URI: {'uri': 'http://bmi.utah.edu/ontologies/hfontology/C0948089', 'label': 'Acute Coronary Syndrome', 'similarity': 1.0}
clinicalFeatureLabel: 'Change in pattern during 24h' - URI: {'uri': 'http://bmi.utah.edu/ontologies/hfontology/T0

In [6]:
from resources.scripts.vis import draw

options = {"Article": "label", "ClinicalFeature": "label", "ClinicalFeatureGroup": "label", "Finding": "title", "LR": "title"}
draw(graph, options,  physics=True)

In [7]:
import requests
import json  

payload = { 'text': 'chest pain', 'similarity': 0.5, 'ontology':'hfo' }

r = requests.post('http://ontomatch.lis.ic.unicamp.br/api/rest/resource', data=json.dumps(payload))
print(r.status_code)

restReturn = r.text.replace('callback(', '')
restReturn = restReturn.replace('})', '}')
print(restReturn)


returnJson = json.loads(restReturn)  
print(returnJson['uri'], returnJson['similarity'])



200
{"uri":"http://bmi.utah.edu/ontologies/hfontology/T0000048","label":"(a pulm-a mitral) duration","similarity":1.0}
http://bmi.utah.edu/ontologies/hfontology/T0000048 1.0


In [8]:
import requests
import json  

payload = { 'text': 'chest pain', 'similarity': 0.5, 'ontology':'hfo' }

r = requests.post('http://ontomatch.lis.ic.unicamp.br/api/rest/resource', data=json.dumps(payload))
print(r.status_code)

restReturn = r.text.replace('callback(', '')
restReturn = restReturn.replace('})', '}')
print(restReturn)


returnJson = json.loads(restReturn)  
print(returnJson['uri'], returnJson['similarity'])




200
{"uri":"http://bmi.utah.edu/ontologies/hfontology/T0000048","label":"(a pulm-a mitral) duration","similarity":1.0}
http://bmi.utah.edu/ontologies/hfontology/T0000048 1.0
